In [1]:
import nltk
import pandas as pd
from nltk.collocations import *
from nltk.metrics import (
    ContingencyMeasures,
    BigramAssocMeasures,
    TrigramAssocMeasures,
    QuadgramAssocMeasures,
)
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

df = pd.read_csv("./Data/Data scientist_London.csv")

df.head()

,Unnamed: 0,Position,City,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data scientist,London,Medical Data Scientist,-1,About Us\n\nAt Sensyne Health we combine techn...,4.0,Sensyne Health\n4.0,"Oxford, England",-1,51 to 200 Employees,2018,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $5 million (USD),-1
1,1,Data scientist,London,Data Scientist,-1,"Data Scientist\n£55,000 – £70,000\n\n\nLocatio...",4.2,Anson McCade\n4.2,"London, England",-1,51 to 200 Employees,2000,Company - Private,Staffing & Outsourcing,Business Services,$1 to $5 million (USD),-1
2,2,Data scientist,London,Biomedical Scientist,-1,Fixed Term Contract\nWe're currently seeking h...,-1.0,NHS Test and Trace,"Royal Leamington Spa, England",-1,-1,-1,-1,-1,-1,-1,-1
3,3,Data scientist,London,Biomedical Scientist,-1,Fixed Term Contract\nWe're currently seeking h...,-1.0,NHS Test and Trace,"Royal Leamington Spa, England",-1,-1,-1,-1,-1,-1,-1,-1
4,4,Data scientist,London,Data Scientist,-1,Fixed Term Contract\nWe're currently seeking D...,-1.0,NHS Test and Trace,"Royal Leamington Spa, England",-1,-1,-1,-1,-1,-1,-1,-1


In [1]:
import pickle
f = open('./resp_file.txt', 'r')
responsibilities = f.read().split('\n')
f = open('./req_file.txt', 'r')
requirements = f.read().split('\n')
f.close()
f = open('./workenv_file.txt', 'r')
conditions = f.read().split('\n')
f.close()
requirements = requirements[:-1]
conditions = conditions[:-1]
responsibilities = responsibilities[:-1]

In [2]:
# n - length of requirements list
n = len(requirements)
# m - length of responsibilities list
m = len(responsibilities)
# l - length of conditions list
l = len(conditions)

In [3]:
import glob
import pandas as pd
filenames = sorted(glob.glob('./Data/*.csv'))
    
li = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [4]:
req_relevance_matrix = pd.read_csv('./Result_req.csv', index_col=None, header=0).drop(columns = ['Unnamed: 0']).to_numpy()
resp_relevance_matrix = pd.read_csv('./Result_resp.csv', index_col=None, header=0).drop(columns = ['Unnamed: 0']).to_numpy()
cond_relevance_matrix = pd.read_csv('./Result_we.csv', index_col=None, header=0).drop(columns = ['Unnamed: 0']).to_numpy()

In [5]:
mask_req = req_relevance_matrix>=0.3
mask_resp = resp_relevance_matrix>=0.27
mask_cond = cond_relevance_matrix>=0.23

In [6]:
import numpy as np
RRD = pd.DataFrame(columns=['requirements_refined','req_idxs', 'responsibilities_refined', 'resp_idxs', 'conditions_refined', 'cond_idxs'])


In [7]:
for i in range(len(mask_req)):
    RRD = RRD.append({'requirements_refined':np.array(requirements)[mask_req[i]],'req_idxs':np.where(req_relevance_matrix[i]>=0.3),
                      'responsibilities_refined':np.array(responsibilities)[mask_resp[i]],'resp_idxs':np.where(resp_relevance_matrix[i]>=0.27),
                      'conditions_refined':np.array(conditions)[mask_cond[i]],'cond_idxs':np.where(cond_relevance_matrix[i]>=0.23)}, ignore_index=True)

In [8]:
final = pd.concat([frame, RRD], axis=1)

In [60]:
final.shape

(10300, 23)

Tri-clustering Overlap

In [11]:

#union of two sets
def union(lst1,lst2):
    return list(set(lst1).union(set(lst2)))

In [12]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [34]:

# build a 3d matrix
import numpy as np
res= np.zeros((n,m,l), dtype=float)
for i in range(len(w)):
    temp= np.zeros((n,m,l), dtype=float)
    for el in reqs[i]:
        for el2 in resps[i]:
            for el3 in conds[i]:
                # if combination (el,el2,el3) is in vacancy's (requirements, responsibilities, conditions), 
                # then add 1 to the (el,el2,el3)-th element of 3d matrix
                temp[int(el)][int(el2)][int(el3)]+=1.0
    # after we iterated over all requirements, responsibilities, conditions sets of a vacancy, 
    # add this 3d adjacency matrix to the r-matrix, but divide by the weight of the vacancy
    # Divide - because the larger the weight is, the more requirements, responsibilities and conditions were exctracted
    # for the vacancy - thus it's less important and more general
    if w[i]!=0:
        res+=temp*1.0/w[i]

res = [[[np.round(el*1.0,2) for el in el2] for el2 in el3] for el3 in res]

NameError: name 'w' is not defined

In [174]:
res = np.array(res)


In [13]:
triplets = []
for i in range(len(w)):
    for el in reqs[i]:
            for el2 in resps[i]:
                for el3 in conds[i]:
                    triplets.append([el,el2,el3])

In [14]:
duplets = []
for i in range(len(w)):
    for el in reqs[i]:
            for el2 in resps[i]:
                duplets.append([el,el2,-1])
for i in range(len(w)):
    for el in resps[i]:
            for el2 in conds[i]:
                duplets.append([-1,el,el2])
for i in range(len(w)):
    for el in reqs[i]:
            for el2 in conds[i]:
                duplets.append([el,-1,el2])

In [15]:
duplets_DF=pd.DataFrame(duplets, columns = ['Req','Resp','Cond'])


In [17]:
duplets_DF

,Req,Resp,Cond
0,5,25,-1
1,5,41,-1
2,5,71,-1
3,5,94,-1
4,5,103,-1
...,...,...,...
5862407,359,-1,39
5862408,365,-1,39
5862409,369,-1,39
5862410,389,-1,39


In [16]:
duplets_DF_2= duplets_DF.groupby(['Req','Resp','Cond']).size().reset_index(name='size')


In [17]:
bi_clusters_resp_x_cond = duplets_DF_2.loc[(duplets_DF_2['Req']==-1)]
bi_clusters_req_x_cond = duplets_DF_2.loc[(duplets_DF_2['Resp']==-1)]
bi_clusters_req_x_resp = duplets_DF_2.loc[(duplets_DF_2['Cond']==-1)]

req_x_cond

In [18]:
import statistics
statistics.median(duplets_DF_2['size'])

22.0

In [19]:
bi_clusters_req_x_cond = bi_clusters_req_x_cond.loc[(bi_clusters_req_x_cond['size']>5)]

bi_clusters_req_x_cond = bi_clusters_req_x_cond.groupby('Req')['Cond'].apply(lambda x: set(x)).reset_index(name='Conds')

bi_clusters_req_x_cond.head(2)

,Req,Conds
0,1,"{3, 10, 15, 19, 25, 33, 39, 40, 41, 43, 50, 56..."
1,2,"{0, 3, 5, 8, 10, 11, 14, 15, 19, 20, 21, 24, 2..."


In [20]:
triplets_DF=pd.DataFrame(triplets, columns = ['Req','Resp','Cond'])
triplets_DF = triplets_DF.groupby(['Req','Resp','Cond']).size().reset_index(name='size')

In [21]:
clusters_req_v1 = []
clusters_cond_v1 = []
clusters_resp_v1 = []
used_reqs = set()
for req in bi_clusters_req_x_cond.values:
    res_cond = set()
    res_req = set()
    res_resp = set()
    if req[0] not in used_reqs:
        el = req
        res_cond = el[1]
        res_req.add(el[0])
        for el2 in bi_clusters_req_x_cond.values:
            if el2[0] not in used_reqs and el2[0] not in res_req:
                    if jaccard_similarity(res_cond,el2[1])>=0.8:
                        res_cond = res_cond.intersection(el2[1])               
                        res_req.add(el2[0])
                        used_reqs.add(el2[0])
                        for cond in res_cond:
                            temp = triplets_DF[(triplets_DF['Req']==el[0]) &(triplets_DF['Cond']==cond)]
                            temp = temp[temp['size']== max(temp['size'])]['Resp']
                            res_resp = res_resp.union(temp)                   
        used_reqs.add(req[0])
        if len(res_req)>1 and len(res_cond)>1:
            clusters_req_v1.append(res_req) 
            clusters_cond_v1.append(res_cond)
            clusters_resp_v1.append(res_resp)

In [22]:
req_x_cond_df = pd.DataFrame({'Req_clusters':clusters_req_v1,'Resp_clusters':clusters_resp_v1, 'Cond_clusters': clusters_cond_v1})

req_x_cond_df_phrases = pd.DataFrame(columns = {'Req_clusters','Cond_clusters'})
for i in range(len(clusters_req_v1)):
    req_x_cond_df_phrases = req_x_cond_df_phrases.append({'Req_clusters':[requirements[int(el)].replace('\n','') for el in clusters_req_v1[i]]
                       ,'Resp_clusters': [responsibilities[int(el)].replace('\n','') for el in clusters_resp_v1[i]],
                        'Cond_clusters': [conditions[int(el)].replace('\n','') for el in clusters_cond_v1[i]]},ignore_index = True)

req_x_cond_df_phrases

,Req_clusters,Cond_clusters,Resp_clusters
0,"[Red Hat Enterprise Linux, Bachelors degree in...","[Flexible Working, We are an equal opportunity...","[Lights out management, maintenance, and admin..."
1,[Ability to communicate clearly and concisely....,"[Pension , Flexible Working, We are an equal o...","[documentation creation, Provide ad hoc operat..."
2,"[Bitbucket, Atlassian products, Experience in ...","[Flexible Working, We are an equal opportunity...","[documentation creation, Provide ad hoc operat..."
3,"[JQuery, Experience delivering to enterprise c...","[Pension , Flexible Working, We are an equal o...","[documentation creation, Provide ad hoc operat..."
4,"[Java, Data science models]","[Pension , Flexible Working, We are an equal o...","[documentation creation, JavaScript libraries,..."
5,"[Android, extensive experience in leading and ...","[Pension , Flexible Working, We are an equal o...","[documentation creation, technical evaluation,..."
6,"[hierarchical models, Computer literacy – part...","[Pension , Flexible Working, We are an equal o...","[documentation creation, Problem Solving, tech..."
7,[Diligently maintains a molecular biology labo...,"[Pension , Flexible Working, We are an equal o...","[documentation creation, Scaled Agile Developm..."
8,[Learning and development tailored to your rol...,"[Pension , Flexible Working, We are an equal o...","[documentation creation, Remain current on inf..."
9,"[OpenShift, Artifactory]","[Flexible Working, We are an equal opportunity...","[documentation creation, automation with Ansib..."


req_x_resp

In [23]:
bi_clusters_req_x_resp = bi_clusters_req_x_resp.loc[bi_clusters_req_x_resp['size']>5]

bi_clusters_req_x_resp = bi_clusters_req_x_resp.groupby('Req')['Resp'].apply(lambda x: set(x)).reset_index(name='Resps')

bi_clusters_req_x_resp.head(2)

,Req,Resps
0,1,"{0, 256, 258, 139, 12, 23, 25, 29, 33, 36, 166..."
1,2,"{0, 5, 6, 12, 13, 14, 19, 21, 23, 25, 29, 32, ..."


In [24]:
clusters_req_v2 = []
clusters_resp_v2 = []
clusters_cond_v2 = []
used_reqs = set()
for req in bi_clusters_req_x_resp.values:
    res_resp = set()
    res_req = set()
    res_cond = set()
    if req[0] not in used_reqs:
        el = req
        res_resp = el[1]
        res_req.add(el[0])
        for el2 in bi_clusters_req_x_resp.values:
            if el2[0] not in used_reqs and el2[0] not in res_req:
                    if jaccard_similarity(res_resp,el2[1])>=0.8:
                        res_resp = res_resp.intersection(el2[1])               
                        res_req.add(el2[0])
                        used_reqs.add(el2[0])
                        for resp in res_resp:
                            temp = triplets_DF[(triplets_DF['Req']==el[0]) &(triplets_DF['Resp']==resp)]
                            temp = temp[temp['size']== max(temp['size'])]['Cond']
                            res_cond = res_cond.union(temp)
        used_reqs.add(req[0])
        if len(res_req)>1 and len(res_resp)>1:
            clusters_req_v2.append(res_req) 
            clusters_resp_v2.append(res_resp)
            clusters_cond_v2.append(res_cond)

In [25]:
req_x_resp_df = pd.DataFrame({'Req_clusters':clusters_req_v2,'Resp_clusters': clusters_resp_v2,'Cond_clusters': clusters_resp_v2})

req_x_resp_df_phrases = pd.DataFrame(columns = {'Req_clusters','Resp_clusters'})
for i in range(len(clusters_req_v2)):
    req_x_resp_df_phrases = req_x_resp_df_phrases.append({'Req_clusters':[requirements[int(el)].replace('\n','') for el in clusters_req_v2[i]]
                       ,'Resp_clusters': [responsibilities[int(el)].replace('\n','') for el in clusters_resp_v2[i]],
                        'Cond_clusters': [conditions[int(el)].replace('\n','') for el in clusters_cond_v2[i]]},ignore_index = True)

req_x_resp_df_phrases

,Resp_clusters,Req_clusters,Cond_clusters
0,"[Implements, supports and maintains Epic clini...","[Python, Experience in data modeling, At least...","[Working from Home, Working Locations:, third ..."
1,"[Implements, supports and maintains Epic clini...","[Strong background in statistics, completed ed...","[Flexible Working, Working Locations:, OUR OPP..."
2,"[Implements, supports and maintains Epic clini...","[knowledge of revision control systems, Active...","[Benefit Conditions:, Working from Home, Flexi..."
3,"[Implements, supports and maintains Epic clini...","[Mobile Testing, Android]","[Benefit Conditions:, Flexible Working, Cycle ..."
4,"[Implements, supports and maintains Epic clini...","[time series forecasting, data pipeline, Dynam...",[We are an equal opportunity employer and we v...
5,"[Implements, supports and maintains Epic clini...","[Disclosure and Barring Service Check, Success...","[Pension , Flexible Working, We are an equal o..."
6,"[Implements, supports and maintains Epic clini...","[Jenkins, deep learning, Making Effective Deci...","[Pension , Flexible Working, We are an equal o..."
7,"[Implements, supports and maintains Epic clini...","[Security Policy generation, specialized train...","[Flexible Working, 4/10 Schedule, Working Loca..."
8,"[Implements, supports and maintains Epic clini...",[Understanding of mobile F2P business practice...,"[Flexible Working, OUR OPPORTUNITY, Working Lo..."
9,"[Implements, supports and maintains Epic clini...","[UIAutomation, experience in Bioinformatics, C...","[Flexible Working, Flexibility in scheduling, ..."


resp_x_cond

In [26]:
bi_clusters_resp_x_cond = bi_clusters_resp_x_cond.loc[(bi_clusters_resp_x_cond['size']>5)]

bi_clusters_resp_x_cond = bi_clusters_resp_x_cond.groupby('Resp')['Cond'].apply(lambda x: set(x)).reset_index(name='Conds')

bi_clusters_resp_x_cond.head(2)

,Resp,Conds
0,0,"{0, 3, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16, 19,..."
1,1,"{0, 3, 10, 11, 14, 15, 19, 20, 21, 25, 33, 39,..."


In [27]:
clusters_resp_v3 = []
clusters_cond_v3 = []
clusters_req_v3 = []
used_resps = set()
for resp in bi_clusters_resp_x_cond.values:
    res_cond = set()
    res_resp = set()
    res_req=set()
    if resp[0] not in used_resps:
        el = resp
        res_cond = el[1]
        res_resp.add(el[0])
        for el2 in bi_clusters_resp_x_cond.values:
            if el2[0] not in used_resps and el2[0] not in res_resp:
                    if jaccard_similarity(res_cond,el2[1])>=0.8:
                        res_cond = res_cond.intersection(el2[1])               
                        res_resp.add(el2[0])
                        used_resps.add(el2[0])
                        for cond in res_cond:
                            temp = triplets_DF[(triplets_DF['Resp']==el[0]) &(triplets_DF['Cond']==cond)]
                            temp = temp[temp['size']== max(temp['size'])]['Req']
                            res_req = res_req.union(temp)
        used_resps.add(resp[0])
        if len(res_resp)>1 and len(res_cond)>1:
            clusters_resp_v3.append(res_resp) 
            clusters_cond_v3.append(res_cond)
            clusters_req_v3.append(res_req)

In [28]:
resp_x_cond_df = pd.DataFrame({'Req_clusters': clusters_req_v3,'Resp_clusters':clusters_resp_v3,'Cond_clusters': clusters_cond_v3})

resp_x_cond_df_phrases = pd.DataFrame(columns = {'Req_clusters','Resp_clusters','Cond_clusters'})
for i in range(len(clusters_resp_v3)):
    resp_x_cond_df_phrases = resp_x_cond_df_phrases.append({'Req_clusters':[requirements[int(el)].replace('\n','') for el in clusters_req_v3[i]],
                                                            'Resp_clusters':[responsibilities[int(el)].replace('\n','') for el in clusters_resp_v3[i]]
                       ,'Cond_clusters': [conditions[int(el)].replace('\n','') for el in clusters_cond_v3[i]]},ignore_index = True)

resp_x_cond_df_phrases

,Resp_clusters,Req_clusters,Cond_clusters
0,"[Implements, supports and maintains Epic clini...","[Replication, working on user-experiences, 4+ ...","[Flexible Working, We are an equal opportunity..."
1,"[first point of contact for web issues, the EC...","[Excellent communication, presentation, interp...","[Working from Home, Flexible Working, Flexibil..."
2,[rigging and enveloping hero to background cre...,"[experience in Bioinformatics, Experience of d...","[Flexible Working, We are an equal opportunity..."
3,[Build BigData systems – work on improving and...,"[Experience in data modeling, experience in Bi...",[We are an equal opportunity employer and we v...
4,"[perform technical administration, Provides ha...","[working on user-experiences, 4+ years of soft...","[Pension , Flexible Working, We are an equal o..."
5,"[Agile, load balancers]",[Good knowledge of video games across multiple...,"[Pension , Flexible Working, We are an equal o..."
6,"[Manage sales tool activation and licenses, As...","[Bachelors Degree, constraint programming, exp...","[Flexible Working, Flexibility in scheduling, ..."
7,"[Selenium, Lead the craftsmanship, availabilit...","[Epic implementation experience, years of expe...","[Pension , Flexible Working, We are an equal o..."
8,[Remain current on information security best p...,"[Excellent communication, presentation, interp...","[Pension , Flexible Working, We are an equal o..."
9,[Recognizes and prioritizes the most important...,"[Experience in data modeling, experience in Bi...","[Flexible Working, able to hire remotely, temp..."


Last

In [60]:
clusters_final_req = []
clusters_final_resp = []
clusters_final_cond = []
for i in range(len(clusters_req_v1)):
    reqs = clusters_req_v1[i]
    resps = clusters_resp_v1[i]
    conds = clusters_cond_v1[i]
    max_sim = -1
    match = -1
    for j in range(len(clusters_req_v2)):
        reqs_2 = clusters_req_v2[j]
        resps_2 = clusters_resp_v2[j]
        conds_2 = clusters_cond_v2[j]
        if jaccard_similarity(reqs,reqs_2)>max_sim:
            match = j
            max_sim = jaccard_similarity(reqs,reqs_2)
    if max_sim>0.2:
        reqs_2 = clusters_req_v2[match]
        resps_2 = clusters_resp_v2[match]
        conds_2 = clusters_cond_v2[match]
        print('reqs')
        print(reqs)
        print(reqs_2)
        print('resps')
        print(set(resps)& set(resps_2))
        print('conds')
        print(set(conds)& set(conds_2))
        clusters_final_req.append(list(union(reqs,reqs_2)))
        clusters_final_resp.append(set(resps)& set(resps_2))
        clusters_final_cond.append(set(conds)& set(conds_2))

reqs
{121, 130, 3, 4, 5, 39, 9, 74, 18, 82, 84, 116, 118, 182, 24, 57, 26, 27, 120, 252}
{34, 3, 66, 5, 39, 9, 410, 17, 18, 50, 84, 116, 118, 178, 88, 181, 26}
resps
{256, 33, 166, 103, 104, 233, 41, 139, 205, 23, 25}
conds
{98, 39, 73, 41, 75, 107, 110, 19, 84}
reqs
{129, 13, 14, 17, 31, 161, 34, 41, 43, 44, 45, 48, 50, 53, 54, 181, 56, 60, 66, 327, 87, 88, 89, 100, 233, 125}
{137, 139, 13, 14, 283, 31, 41, 171, 44, 45, 54, 56, 185, 57, 196, 197, 200, 87, 89, 350, 100, 240, 251}
resps
{256, 33, 36, 249, 103, 233, 41, 205, 25, 217, 94}
conds
{97, 98, 3, 68, 70, 39, 71, 41, 10, 73, 75, 50, 19}
reqs
{16, 277}
{277, 183}
resps
{256, 103, 41, 23, 249, 25}
conds
{3, 70, 71, 39, 41, 10, 75, 19, 84}
reqs
{392, 38}
{179, 38}
resps
{256, 67, 103, 169, 41, 203, 46, 94, 184, 217, 25}
conds
{0, 3, 10, 11, 15, 16, 19, 20, 24, 25, 29, 33, 39, 40, 41, 42, 43, 45, 50, 54, 56, 58, 68, 71, 72, 73, 75, 76, 78, 82, 84, 86, 88, 97, 98, 99, 101, 107, 110}
reqs
{360, 217, 186, 205}
{360, 217, 186, 205}
resps

In [61]:
final_clusters_df = pd.DataFrame({'Req_clusters': clusters_final_req,'Resp_clusters':clusters_final_resp,'Cond_clusters': clusters_final_cond})

final_clusters_df_df_phrases = pd.DataFrame(columns = {'Req_clusters','Resp_clusters','Cond_clusters'})
for i in range(len(final_clusters_df)):
    final_clusters_df_df_phrases = final_clusters_df_df_phrases.append({'Req_clusters':[requirements[int(el)].replace('\n','') for el in clusters_final_req[i]],
                                                            'Resp_clusters':[responsibilities[int(el)].replace('\n','') for el in clusters_final_resp[i]]
                       ,'Cond_clusters': [conditions[int(el)].replace('\n','') for el in clusters_final_cond[i]]},ignore_index = True)

final_clusters_df_df_phrases

,Resp_clusters,Req_clusters,Cond_clusters
0,"[documentation creation, Provide ad hoc operat...","[Atlassian products, Experience in data modeli...","[Working from Home, Working Locations:, third ..."
1,"[documentation creation, Provide ad hoc operat...","[JQuery, knowledge of revision control systems...","[Benefit Conditions:, Working from Home, Flexi..."
2,"[documentation creation, fast paced environmen...","[Java, Data science models, Bachelor''s Degree...","[Flexible Working, 4/10 Schedule, OUR OPPORTUN..."
3,"[documentation creation, Remain current on inf...",[Learning and development tailored to your rol...,"[Pension , Flexible Working, We are an equal o..."
4,"[documentation creation, Palo Alto Networks ha...","[Palo Alto Networks NGFW, Palo Alto Networks (...","[Flexible Working, We are an equal opportunity..."
5,"[documentation creation, Provide ad hoc operat...","[specialized training, automation, Advanced kn...","[Pension , Flexible Working, We are an equal o..."
6,"[documentation creation, Provide ad hoc operat...","[Excellent communication, presentation, interp...","[Benefit Conditions:, Working from Home, Flexi..."


In [62]:
final_clusters_df_df_phrases.to_csv('tri-clusters_14.csv')

Tricluster Box

In [37]:
reqs = []
resps = []
conds = []
for el in weights.values:
    req = list(el[0][0])
    resp = list(el[1][0])
    cond = list(el[2][0])
    reqs.append(req)
    resps.append(resp)
    conds.append(cond)

In [38]:

# build a 3d matrix
import numpy as np
res= np.zeros((n,m,l), dtype=float)
for i in range(len(w)):
    if w[i]>0:
        temp= np.zeros((n,m,l), dtype=float)
        for el in reqs[i]:
            for el2 in resps[i]:
                for el3 in conds[i]:
                    # if combination (el,el2,el3) is in vacancy's (requirements, responsibilities, conditions), 
                    # then add 1 to the (el,el2,el3)-th element of 3d matrix
                    temp[int(el)][int(el2)][int(el3)]+=1.0
        # after we iterated over all requirements, responsibilities, conditions sets of a vacancy, 
        # add this 3d adjacency matrix to the r-matrix, but divide by the weight of the vacancy
        # Divide - because the larger the weight is, the more requirements, responsibilities and conditions 
        # were exctracted
        # for the vacancy - thus it's less important and more general
        res+=temp*1.0/w[i]

res = [[[np.round(el*1.0,2) for el in el2] for el2 in el3] for el3 in res]

In [39]:
res = np.array(res)

In [43]:
a_ipp = np.zeros(n,dtype=float)
a_jpp = np.zeros(m,dtype=float)
a_kpp = np.zeros(l,dtype=float)
for ii in range(n):
    a_ipp[ii]=sum(sum(res[ii][:][:]))
for jj in range(m):
    a_jpp[jj]=sum(sum(res[:][jj][:]))
for kk in range(l):
    a_kpp[kk]=sum(sum(res[:][:][kk]))
    
a_ijk = np.zeros((n,m,l),dtype=float)
for ii in range(n):
    for jj in range(m):
        for kk in range(l):
            a_ijk[ii][jj][kk] = round(a_ipp[ii]*a_jpp[jj]*a_kpp[kk]*1.0/ sum_res**2,2)

In [44]:
res = res - a_ijk


In [45]:
print(len(res[res>0]))
print(len(res[res<0]))

1946328
1255686


In [41]:
sum_res = 0
for el in res:
    for el2 in el:
        for el3 in el2:
            if el3>0:
                sum_res+=el3

In [42]:
sum_res

4189240.0099864937

In [46]:
def sum_VWU(V,W,U,res):
    r = 0
    for el in V:
        for el2 in W:
                for el3 in U:
                    r+=res[el][el2][el3]
    return r

# sum elements of 3d matrix on i,W,U places, where W,U - arrays of indexes of responsibilities, 
# conditions, i - i_th requirement
def sum_iWU(i,W,U,res):
    r = 0
    for el in W:
        for el2 in U:
            r+=res[i][el][el2]
    return r

# sum elements of 3d matrix on i,W,U places, where V,U - arrays of indexes of requirements, 
# conditions, j - j_th responsibility

def sum_jVU(V,j,U,res):
    r = 0
    for el in V:
        for el2 in U:
            r+=res[el][j][el2]
    return r

# sum elements of 3d matrix on V,W places, where V,W - arrays of indexes of requirements, 
# responsibilities, k - k_th condition

def sum_kVW(V,W,k,res):
    r = 0
    for el in V:
        for el2 in W:
            r+=res[el][el2][k]
    return r

In [47]:
import numpy
def del_(r,V,W,U,row,col,xray):
    # length of requirements array
    n = len(r)
    # length of responsibilities array
    m = len(r[0])
    # length of rconditions array
    l = len(r[0][0])
    # nv,nw,nu - lengths of current tri-box cluster
    nv=len(V)
    nw=len(W)
    nu= len(U)
    rVWU = sum_VWU(V,W,U,r)
        
    if nw*nv*nu == 0:
        print('All sets in a box must be non-empty!')
        return val,z
    
    if len(col)*len(row)*len(xray) != 0:
        print('Either row or col or xray must be 0 at changing the box')
        return val,z
    
    if len(col) + len(row) + len(xray) == 0:
        print('Either row or col or xray must be not 0 at changing the box')
        return val,z
    
    # if row=requirement is to be added
    if len(row) > 0:
        row = row[0]
        # sr sum of elements for the row
        sr= sum_iWU(row,W,U,r)
        it= 2*sr*rVWU
        #if row is in set already, we need to delete it, so z=-1
        if row in V:
            z=- 1
        else:
            z=1
        # val - the difference at cluster data-scatter upon adding or removing row to the box
        if sr<0.0: 
            val = -1
        else:
            val=z*it + sr ** 2 - z*rVWU**2 / nv
            if nv + z == 0:
                val=- 1
            else:
                val=val / (nw*nu*(nv + z))
    else:
        # if xray=condition is to be added
        if len(xray) > 0:
            xray = xray[0]
            # sx sum of elements for the xray
            sx=sum_kVW(V,W,xray,r)
            it=2*sx*rVWU
            #if xray is in set already, we need to delete it, so z=-1
            if xray in U:
                z=- 1
            else:
                z=1
            if sx<0.0:
                val=-1
            else:# val - the difference at cluster data-scatter upon adding or removing xray to the box
                val=z*it + sx ** 2 - z*rVWU**2 / nu
                if nu + z == 0:
                    val=- 1
                else:
                    val=val / (nv*nw*(nu + z))
        else:
            # if col=responsibility is to be added
            if len(col) > 0:
                col = col[0]
                # sc sum of elements for the column
                sc=sum_jVU(V,col,U,r)
                it=2*sc*rVWU
                if col in W:
                    z=- 1
                else:
                    z=1
                if sc<0.0:
                    val = -1
                else:# val - the difference at cluster data-scatter upon adding or removing col to the box
                    val=z*it + sc ** 2 - z*rVWU**2 / nw
                    if nw + z == 0:
                        val=- 1
                    else:
                        val=val / (nv*nu*(nw + z))
            else:
                print('No col or row or xray supplied!!!')
                return val,z
    return val,z

In [48]:
def nbox(r,V,W,U):
    
    # length of requirements array
    n = len(r)
    # length of responsibilities array
    m = len(r[0])
    # length of rconditions array
    l = len(r[0][0])
    # nv,nw,nu - lengths of current tri-box cluster
    nv=len(V)
    nw=len(W)
    nu= len(U)
    
    la=sum_VWU(V,W,U,r) / (nv*nw*nu)
    cont=nv*nw*nu*la ** 2
    
    
    # find the largest delta at adding, row or col or xray
    
    row=[]
    cnr=- 1000
    for ii in range(n):
        v,z=del_(r,V,W,U,[ii],[],[])
        if v > cnr:
            cnr=v
            zr=z
            row=ii
    col=[]
    cnc=- 1000
    for jj in range(m):
        v,z=del_(r,V,W,U,[],[jj],[])
        if v > cnc:
            cnc=v
            zc=z
            col=jj
    xray=[]
    cnx=- 1000
    for kk in range(l):
        v,z=del_(r,V,W,U,[],[],[kk])
        if v > cnx:
            cnx=v
            zx=z
            xray=kk
            
    # if the largest delta is for the row          
    if cnr==max([cnc,cnr,cnx]):
        if cnr > 0:
            if zr > 0:
                Vn=union(V,[row])
            else:
                Vn = list(set(V)-set([row]))
            Wn=W
            Un=U
            dcn=cnr
        else:
            Vn=V
            Wn=W
            Un=U
            dcn=0
    else:
        # if the largest delta is for the col
        if cnc == max([cnc,cnr,cnx]):
            if cnc > 0:
                if zc > 0:
                    Wn=union(W,[col])
                    
                else:
                    Wn=list(set(W)-set([col]))
                    
                Vn=V
                Un=U
                dcn=cnc
            else:
                Vn=V
                Wn=W
                Un=U
                dcn=0
        else:
            # if the largest delta is for the xray
            if cnx == max([cnc,cnr,cnx]):
                if cnx > 0:
                    if zx > 0:
                        Un=union(U,[xray])
                    else:
                        Un=list(set(U)-set([xray]))
                    Wn=W
                    Vn=V
                    dcn=cnx
                else:
                    Vn=V
                    Wn=W
                    Un=U
                    dcn=0
    # return themodified box and contribution of adding the element to datascatter
    return Vn,Wn,Un,dcn

In [49]:
import numpy as np
def brc_2(r,row,col,xray):
    r = np.array(r)
    counf=0
    n = len(r)
    m = len(r[0])
    l = len(r[0][0])
    V=[row]
    W=[col]
    U=[xray]
    # average of tri-cluster
    la = sum_VWU(V,W,U,r) / (len(V)*len(W)*len(U))
    # contribution to datascatter
    co = (la ** 2)*(len(V)*len(W)*len(U))
    dd =1 
    # repeat cycle till adding an element to it makes a pisitive difference (Mirkin&Kramarenko article)
    while (dd > 0):
            
            V,W,U,dd=nbox(r,V,W,U)
            if row not in V or col not in W or xray not in U:
                return [],[],[],0,0,0
            co=co + dd
            counf=counf + 1
    # contribution - sum of curren cluster divided by the general sum of matrix r
    contribution = sum_VWU(V,W,U,r)*1.0/sum_res
    print('counf')
    print(counf)
    print('sum_VWU')
    print(sum_VWU(V,W,U,r))
    print('V')
    print(V)
    print('W')
    print(W)
    print('U')
    print(U)
    print('contribution')
    print(contribution)
    return V,W,U,la,co,contribution

In [59]:
used_V = []
used_W = []
used_U = []
clusters = []
for ii in range(n):
    for jj in range(m):
        for kk in range(l):
            if res[ii][jj][kk]>0.0 and ii not in used_V and jj not in used_W and kk not in used_U:
                # we start at a not null element in our r-maxtrix
                # we start at an element, that is not already in any cluster as clusters don't intersect
                # we get the tri-cluster
                V,W,U,la,co,contribution = brc_2(res,ii,jj,kk)
                # we check if it's in cluster set already
                temp = [set(V),set(W),set(U)]
                if temp not in clusters and len(V)>0:
                    clusters.append(temp)
                    print('new_cluster')
                    print(temp)
                    for el in V:
                        used_V.append(el)
                    for el in W:
                        used_W.append(el)
                    for el in U:
                        used_W.append(el)
    print('Working on II num + ' + str(ii))

Working on II num + 0
Working on II num + 1
Working on II num + 2


KeyboardInterrupt: 

In [ ]:
tri_clusters_df_v1 = pd.DataFrame(columns = ['Requirements','Responsibilities',
                              'Conditions'])
for i in range(len(clusters)):
    tri_clusters_df_v1 = tri_clusters_df_v1.append({'Requirements':[req_picked[el].replace('\n','') for el in clusters[i][0]],
                                                'Responsibilities':[resp_picked[el].replace('\n','') for el in clusters[i][1]],
                                               'Conditions':[cond_picked[el].replace('\n','') for el in clusters[i][2]]}, ignore_index = True)

In [ ]:
tri_clusters_df_v1.head()


In [ ]:
tri_clusters_df_v1.to_csv('tri-clusters_first_v1.csv')

Tricluster Box on updated matrix

In [51]:
import numpy
def del_(r,V,W,U,row,col,xray):
    # length of requirements array
    n = len(r)
    # length of responsibilities array
    m = len(r[0])
    # length of rconditions array
    l = len(r[0][0])
    # nv,nw,nu - lengths of current tri-box cluster
    nv=len(V)
    nw=len(W)
    nu= len(U)
    rVWU = sum_VWU(V,W,U,r)
        
    if nw*nv*nu == 0:
        print('All sets in a box must be non-empty!')
        return val,z
    
    if len(col)*len(row)*len(xray) != 0:
        print('Either row or col or xray must be 0 at changing the box')
        return val,z
    
    if len(col) + len(row) + len(xray) == 0:
        print('Either row or col or xray must be not 0 at changing the box')
        return val,z
    
    # if row=requirement is to be added
    if len(row) > 0:
        row = row[0]
        # sr sum of elements for the row
        sr= sum_iWU(row,W,U,r)
        it= 2*sr*rVWU
        #if row is in set already, we need to delete it, so z=-1
        if row in V:
            z=- 1
        else:
            z=1
        # val - the difference at cluster data-scatter upon adding or removing row to the box
        if sr<0.0: 
            val = -1
        else:
            val=z*it + sr ** 2 - z*rVWU**2 / nv
            if nv + z == 0:
                val=- 1
            else:
                val=val / (nw*nu*(nv + z))
    else:
        # if xray=condition is to be added
        if len(xray) > 0:
            xray = xray[0]
            # sx sum of elements for the xray
            sx=sum_kVW(V,W,xray,r)
            it=2*sx*rVWU
            #if xray is in set already, we need to delete it, so z=-1
            if xray in U:
                z=- 1
            else:
                z=1
            if sx<0.0:
                val=-1
            else:# val - the difference at cluster data-scatter upon adding or removing xray to the box
                val=z*it + sx ** 2 - z*rVWU**2 / nu
                if nu + z == 0:
                    val=- 1
                else:
                    val=val / (nv*nw*(nu + z))
        else:
            # if col=responsibility is to be added
            if len(col) > 0:
                col = col[0]
                # sc sum of elements for the column
                sc=sum_jVU(V,col,U,r)
                it=2*sc*rVWU
                if col in W:
                    z=- 1
                else:
                    z=1
                if sc<0.0:
                    val = -1
                else:# val - the difference at cluster data-scatter upon adding or removing col to the box
                    val=z*it + sc ** 2 - z*rVWU**2 / nw
                    if nw + z == 0:
                        val=- 1
                    else:
                        val=val / (nv*nu*(nw + z))
            else:
                print('No col or row or xray supplied!!!')
                return val,z
    return val,z

In [52]:
def nbox(r,V,W,U):
    
    # length of requirements array
    n = len(r)
    # length of responsibilities array
    m = len(r[0])
    # length of rconditions array
    l = len(r[0][0])
    # nv,nw,nu - lengths of current tri-box cluster
    nv=len(V)
    nw=len(W)
    nu= len(U)
    
    la=sum_VWU(V,W,U,r) / (nv*nw*nu)
    cont=nv*nw*nu*la ** 2
    
    
    # find the largest delta at adding, row or col or xray
    
    row=[]
    cnr=- 1000
    for ii in range(n):
        v,z=del_(r,V,W,U,[ii],[],[])
        if v > cnr:
            cnr=v
            zr=z
            row=ii
    col=[]
    cnc=- 1000
    for jj in range(m):
        v,z=del_(r,V,W,U,[],[jj],[])
        if v > cnc:
            cnc=v
            zc=z
            col=jj
    xray=[]
    cnx=- 1000
    for kk in range(l):
        v,z=del_(r,V,W,U,[],[],[kk])
        if v > cnx:
            cnx=v
            zx=z
            xray=kk
            
    # if the largest delta is for the row          
    if cnr==max([cnc,cnr,cnx]):
        if cnr > 0:
            if zr > 0:
                Vn=union(V,[row])
            else:
                Vn = list(set(V)-set([row]))
            Wn=W
            Un=U
            dcn=cnr
        else:
            Vn=V
            Wn=W
            Un=U
            dcn=0
    else:
        # if the largest delta is for the col
        if cnc == max([cnc,cnr,cnx]):
            if cnc > 0:
                if zc > 0:
                    Wn=union(W,[col])
                    
                else:
                    Wn=list(set(W)-set([col]))
                    
                Vn=V
                Un=U
                dcn=cnc
            else:
                Vn=V
                Wn=W
                Un=U
                dcn=0
        else:
            # if the largest delta is for the xray
            if cnx == max([cnc,cnr,cnx]):
                if cnx > 0:
                    if zx > 0:
                        Un=union(U,[xray])
                    else:
                        Un=list(set(U)-set([xray]))
                    Wn=W
                    Vn=V
                    dcn=cnx
                else:
                    Vn=V
                    Wn=W
                    Un=U
                    dcn=0
    # return themodified box and contribution of adding the element to datascatter
    return Vn,Wn,Un,dcn

In [53]:
import numpy as np
def brc(r,row,col,xray):
    counf=0
    n = len(r)
    m = len(r[0])
    l = len(r[0][0])
    r = np.array(r)
    ff=0
    
    if len(row) > 0:
        V=row
        row = int(row[0])
        try:
            temp = np.where(r[row][:][:]>0.0)
            W = [temp[0][0]]
            try:
                U= [temp[1][0]]
            except:
                ff=1
                return V,W,[],0,0,0
        except:
            ff=1
            return V,[],[],0,0,0
        
    if ff==0:
        # la = the average of tricluster
        la=sum_VWU(V,W,U,r) / (len(V)*len(W)*len(U))
        co=(la ** 2)*(len(V)*len(W)*len(U))
        dd=1
        while (dd > 0):
            V,W,U,dd=nbox(r,V,W,U)
            co=co + dd
            counf=counf + 1
            if counf%20==0:
                print('dd')
                print(dd)
                print('counf')
                print(counf)
                print('sum_VWU')
                print(sum_VWU(V,W,U,r))
                print('V')
                print(V)
                print('W')
                print(W)
                print('U')
                print(U)
            if counf>=1000:
                print('no hope')
                break
    contribution_to_sum = sum_VWU(V,W,U,r)*1.0/sum_res
    return V,W,U,la,co,contribution_to_sum

In [55]:
V=[]
W=[]
U = []
clusters_V=[]
clusters_W=[]
clusters_U=[]
la=0
co=0
temp_cond_picked = conditions
temp_req_picked = requirements
temp_resp_picked = responsibilities
temp_res = res.copy()
bi_clusters_df = pd.DataFrame(columns = ['Requirements','Responsibilities',
                              'Conditions'])
i = 0
c_gen=0
while len(temp_cond_picked)*len(temp_req_picked)!=0:
    V=[]
    W=[]
    U = []
    print(len(temp_res))
    print(len(temp_res[0]))
    print(len(temp_res[0][0]))
    V,W,U,la,co,cont_percent = brc(temp_res,[0],[],[])
    if len(W)==0 or len(U) == 0:
        temp_res = np.delete(temp_res,0, axis=0)
        temp_req_picked = np.delete(temp_req_picked, 0)   
        print('empty cluster')
        print('contribution')
        print(cont_percent)
        c_gen+=cont_percent

    else:
        print('Contribution of cluster '+ str(round(cont_percent,4)))
        clusters_V.append(V)
        clusters_W.append(W)
        clusters_U.append(U)
        bi_clusters_df = bi_clusters_df.append({'Requirements':[temp_req_picked[el].replace('\n','') for el in clusters_V[i]],
                                                'Responsibilities':[temp_resp_picked[el].replace('\n','') for el in clusters_W[i]],
                                               'Conditions':[temp_cond_picked[el].replace('\n','') for el in clusters_U[i]]}, ignore_index = True)
        temp_res = np.delete(np.delete(np.delete(temp_res, U,axis =2), W,axis =1),V, axis=0)
        temp_req_picked = np.delete(temp_req_picked, V)
        temp_resp_picked = np.delete(temp_resp_picked, W)
        temp_cond_picked = np.delete(temp_cond_picked, U)
        c_gen+=cont_percent
        i+=1

411
261
111
empty cluster
contribution
0
410
261
111
dd
635608.621077034
counf
20
sum_VWU
45807.79999999996
V
[145, 113, 147, 137, 42, 395, 382]
W
[0, 256, 94, 36, 103, 233, 25, 41]
U
[3, 19, 71, 39, 41, 10]
dd
692227.8191045031
counf
40
sum_VWU
225938.55000000002
V
[382, 392, 137, 42, 395, 362, 175, 47, 145, 113, 147, 52, 152, 30, 223]
W
[0, 256, 33, 36, 249, 103, 104, 233, 41, 139, 12, 23, 25, 29, 94]
U
[97, 3, 71, 39, 41, 10, 105, 40, 82, 19, 84]
dd
490823.8862020755
counf
60
sum_VWU
521357.9700000011
V
[130, 392, 137, 395, 267, 145, 147, 214, 152, 30, 159, 223, 357, 296, 42, 362, 175, 47, 113, 52, 309, 382]
W
[0, 256, 139, 12, 23, 25, 29, 94, 33, 36, 38, 103, 104, 233, 41, 167, 169, 166, 46, 184, 249]
U
[3, 71, 10, 76, 82, 19, 84, 97, 33, 39, 40, 41, 105, 107, 42, 50, 54, 56]
dd
197783.3077082582
counf
80
sum_VWU
868779.8300000007
V
[130, 387, 392, 137, 395, 267, 13, 335, 145, 147, 404, 214, 152, 30, 159, 223, 357, 296, 42, 299, 362, 175, 47, 113, 52, 309, 382]
W
[0, 256, 258, 195,

empty cluster
contribution
0
119
133
22
empty cluster
contribution
0
118
133
22
empty cluster
contribution
0
117
133
22
empty cluster
contribution
0
116
133
22
empty cluster
contribution
0
115
133
22
empty cluster
contribution
0
114
133
22
empty cluster
contribution
0
113
133
22
empty cluster
contribution
0
112
133
22
empty cluster
contribution
0
111
133
22
empty cluster
contribution
0
110
133
22
empty cluster
contribution
0
109
133
22
empty cluster
contribution
0
108
133
22
empty cluster
contribution
0
107
133
22
empty cluster
contribution
0
106
133
22
empty cluster
contribution
0
105
133
22
empty cluster
contribution
0
104
133
22
empty cluster
contribution
0
103
133
22
empty cluster
contribution
0
102
133
22
empty cluster
contribution
0
101
133
22
empty cluster
contribution
0
100
133
22
empty cluster
contribution
0
99
133
22
empty cluster
contribution
0
98
133
22
empty cluster
contribution
0
97
133
22
empty cluster
contribution
0
96
133
22
empty cluster
contribution
0
95
133
22
empty

In [57]:

bi_clusters_df

,Requirements,Responsibilities,Conditions
0,"[automation, Excellent communication, presenta...","[Implements, supports and maintains Epic clini...","[Flexible Working, We are an equal opportunity..."
1,"[Experience in data modeling, Knowledge of mo...","[firewalls, Supporting particularly iOS operat...","[Pension , Work remotely:, Weekends, 25 days’ ..."
2,"[Strong background in statistics, PostgreSQL, ...",[Develop backend services that expand the capa...,"[subsidised gym membership, Maternity and Pate..."
3,[Routing],"[Juniper Mist, Routing]",[401(k) matching]
4,"[Support Disaster Recovery, Disaster Recovery]","[Support Disaster Recovery, High Availability ...","[May require unscheduled after-hours work, Tui..."
5,"[Browser Stack, bootstrapping, mercurial]",[Browser Stack],[Cycle 2 Work Scheme]
6,"[Reliable, with high work standards and the wi...",[Consistency of terminology and imagery.],"[massage therapy, Summer and Christmas parties..."
7,[Ability to communicate clearly and concisely....,"[account & item set up and management, order p...","[Casual dress, Casual Dress, Salary: per year]"
8,"[Bitbucket, Strong attention to detail]","[Identifying, mitigating and resolving root ca...","[able to hire remotely, fixed term contract]"
9,"[Java, NoSQL]","[Agile, Java]","[in-year bonus scheme, temporarily due to COVI..."


In [58]:
bi_clusters_df.to_csv('tri-clusters_box_v1.csv')